In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 512 # chosen for optimum results and training time
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 2x faster
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # 4bit for 405b!
    "unsloth/Mistral-Small-Instruct-2409",     # Mistral 22b 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!

    "unsloth/Llama-3.2-1B-bnb-4bit",           # NEW! Llama 3.2 models
    "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    "unsloth/Llama-3.2-3B-bnb-4bit",
    "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",

    "unsloth/Llama-3.3-70B-Instruct-bnb-4bit" # NEW! Llama 3.3 70B!
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 is optimized
    bias = "none",    # "none" is optimized
    # "unsloth" uses 30% less VRAM, fits 2x larger batch sizes
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # rank stabilized LoRA is set to false
    loftq_config = None, # And L
)

Unsloth 2025.4.7 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


We are using the google colab drive to save our adapters 

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from datasets import load_dataset
txt_path = '/content/drive/MyDrive/HPML_Project/dataset/finance_corpus.txt'

# load the finance_corpus.txt
ds = load_dataset(
    "text",
    data_files={"train": txt_path},
    split="train",
)
ds = ds.filter(lambda x: x["text"].strip() != "")

print(f"Loaded {len(ds)} examples; sample text:")
print(ds[0]["text"][:200].replace("\n"," "), "…")

Generating train split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/441891 [00:00<?, ? examples/s]

Loaded 227699 examples; sample text:
Article 1: Warren Buffett Autographed Books To Help Charity: Here's How You Can Get Legendary Investor's Signature …


In [7]:
split = ds.train_test_split(test_size=0.10, seed=42)
train_ds = split["train"]
val_ds   = split["test"]

In [8]:
import re
from datasets import Dataset

articles, buf = [], []

with open(txt_path, encoding="utf‑8") as f:
    for line in f:
        # new article header
        if re.match(r"^Article\s+\d+:", line):
            if buf:
                articles.append(" ".join(buf).strip())
                buf = []
        buf.append(line.strip())
    if buf:
        articles.append(" ".join(buf).strip())

print("Total articles:", len(articles))
ds = Dataset.from_dict({"text": articles})

Total articles: 9285


In [9]:
split = ds.train_test_split(test_size=0.10, seed=42)
train_ds = split["train"]
val_ds   = split["test"]

In [10]:
train_ds

Dataset({
    features: ['text'],
    num_rows: 8356
})

In [11]:
for text in ds["text"][:10]:
    toks = tokenizer(text, add_special_tokens=False)
    print(len(toks["input_ids"]), "tokens")

670 tokens
799 tokens
696 tokens
554 tokens
797 tokens
813 tokens
728 tokens
757 tokens
962 tokens
974 tokens


In [12]:
from transformers import TrainerCallback
import math

class PerplexityCallback(TrainerCallback):
    def on_evaluate(self, args, state, control, metrics=None, **kwargs):
        if metrics is not None and "eval_loss" in metrics:
            ppl = math.exp(metrics["eval_loss"])
            print(f"Eval perplexity: {ppl:.2f}")

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_ds,
    eval_dataset     = val_ds,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,

    # data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks = [PerplexityCallback()],
    dataset_num_proc = 2,
    packing = True, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 32,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 10, 
        # max_steps = 60,
        eval_strategy = "epoch",
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # dont report to WandB lol, it asks for API KEY
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/8356 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/929 [00:00<?, ? examples/s]

Unsloth: Hugging Face's packing is currently buggy - we're disabling it for now!


In [14]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|>Article 3263: Breaking News: Dow, S&amp;P, Nasdaq Futures Tumble Ahead of Big Earnings Week  April 21 - U.S. equity futures declined on Monday, as investors returned from the holiday weekend facing renewed concerns over U.S.-China trade tensions and a packed earnings calendar.  Nasdaq 100 futures slid about 1.2%, while Dow Jones Industrial Average and S&P 500 futures dropped nearly 0.9% and 1.1%, respectively. The three major indices also ended last week lower, notching their third loss in four weeks.  Weighing on sentiment was UnitedHealth (NYSE:UNH), which fell more than 22% on Thursday after cutting its full-year outlook and reporting underwhelming earnings. In contrast, Eli Lilly (NYSE:LLY) surged 14% following positive late-stage trial results for its experimental weight loss drug, orforglipron.  The ongoing lack of progress in direct U.S.-China trade talks, along with comments from Chicago Fed President Austan Goolsbee warning of potential economic slowdown from

In [15]:
trainer.train_dataset

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 8356
})

In [16]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,356 | Num Epochs = 10 | Total steps = 2,620
O^O/ \_/ \    Batch size per device = 32 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (32 x 1 x 1) = 32
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Epoch,Training Loss,Validation Loss
1,2.138800,2.121710
2,1.971000,2.005927
3,1.891100,1.946927
4,1.835100,1.912961
5,1.853700,1.889602
6,1.823500,1.873529
7,1.747900,1.861233
8,1.642000,1.853765
9,1.833900,1.848979
10,1.853200,1.847602


Unsloth: Will smartly offload gradients to save VRAM!
Eval perplexity: 8.35
Eval perplexity: 7.43
Eval perplexity: 7.01
Eval perplexity: 6.77
Eval perplexity: 6.62
Eval perplexity: 6.51
Eval perplexity: 6.43
Eval perplexity: 6.38
Eval perplexity: 6.35
Eval perplexity: 6.34


In [17]:
FastLanguageModel.for_inference(model)

def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # generate 
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


print(answer("How is starbucks doing?"))

Starbucks Corporation (NASDAQ:SBUX) has seen its share price fall by 11% over the last month, which may not be a good sign for investors. However, it's important to consider whether this decline was caused by fundamental changes in earnings or if there are other factors that could have been responsible behind these movements. In many cases, companies can see their stock prices move up and down based on how much they pay out as dividends compared with what shareholders receive from new capital raised through equity issuance.
Check out our latest analysis for Starbucks Corporation NasdaqGS:SBSH Earnings and Revenue Growth March 25th 2025 The


In [18]:
save_path = "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_7"  

# Save LoRA adapter + config
trainer.save_model(save_path)

# Save tokenizer files
tokenizer.save_pretrained(save_path)

print(" Saved adapters + tokenizer to", save_path)

 Saved adapters + tokenizer to /content/drive/MyDrive/HPML_Project/copy_unsloth_a100_7


## Inference


In [19]:


import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# Load the same 4-bit base + tokenizer we fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# Load the fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_7",     # folder where we saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # generate
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


print(answer("How is starbucks stock doing?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Starbucks (SBUX) shares are trading up 1.5% at $68.50 as of this writing, following a strong earnings report and guidance that was well-received by Wall Street analysts. The company’s revenue beat expectations for the first quarter while its adjusted EPS came in above consensus estimates. Additionally, management provided positive outlooks on both top-line growth and operating expenses, which could support future share price appreciation.
The recent market action has been driven primarily by concerns around tariffs impacting g

In [22]:
import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# Load the same 4-bit base + tokenizer we fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# Load the fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_7",     # folder where we saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # generate
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


print(answer("What is the best performing stock?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
The answer depends on your investment goals. If you are looking for a high-yielding dividend-paying stock, then investing in Apple (NASDAQ:AAPL) would be an excellent choice as it currently yields 4%. However, if you want to invest with more risk tolerance and higher potential returns than AAPL can offer, then there may not be any better option available.
Apple’s current share price of $144.50 per share implies that its forward P/E ratio stands at about 27 times earnings estimates over the next year. This indicates that invest

In [21]:


import torch
from peft import prepare_model_for_kbit_training, PeftModel
from unsloth import FastLanguageModel

# Load the same 4-bit base + tokenizer we fine-tuned on
base, tokenizer = FastLanguageModel.from_pretrained(
    model_name     = "unsloth/Llama-3.2-1B-bnb-4bit",  # base
    max_seq_length = 128,
    dtype          = torch.float16,                   # or None for auto
    load_in_4bit   = True,
    device_map     = "auto",
)

# ensure pad/eos tokens are set
tokenizer.pad_token = tokenizer.eos_token
base.config.pad_token_id = tokenizer.pad_token_id
base.config.use_cache      = True

# Patch for QLoRA / k-bit adapters
base = prepare_model_for_kbit_training(base)

# Load the fine-tuned LoRA adapters
model = PeftModel.from_pretrained(
    base,
    "/content/drive/MyDrive/HPML_Project/copy_unsloth_a100_7",     # folder where we saved adapters + tokenizer
    device_map="auto",          # shard onto GPU automatically
)

# model.eval()
FastLanguageModel.for_inference(model)

# Inference helper
def answer(prompt: str,
           max_new_tokens: int = 128,
           temperature: float    = 0.2,
           top_p: float          = 0.7,
           repetition_penalty: float = 1.2,
           no_repeat_ngram_size: int = 3):
    # tokenize
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(model.device)

    input_ids = inputs["input_ids"]

    # generate
    outputs = model.generate(
        **inputs,
        max_new_tokens       = max_new_tokens,
        temperature          = temperature,
        top_p                = top_p,
        do_sample            = True,
        repetition_penalty   = repetition_penalty,
        no_repeat_ngram_size = no_repeat_ngram_size,
        eos_token_id         = tokenizer.eos_token_id,
        pad_token_id         = tokenizer.pad_token_id,
        early_stopping       = True,
    )

    # strip off prompt‐tokens and decode only the new ones
    gen_ids = outputs[0][ input_ids.shape[-1] : ]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


print(answer("What is the news on Lockheed Martin?"))

==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
The company has been in a bit of turmoil lately, with its stock price falling by more than 20% over just two days. This decline comes after reports that it may be considering layoffs and other cost-cutting measures to deal with an expected $1 billion loss from tariffs.
The recent developments have also led some analysts to question whether this could impact future earnings forecasts for the aerospace giant. While there are still many details about what’s happening at Lockheed Martin right now, investors will want to keep tabs 